# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,26.35,74,99,0.16,PN,1708924268
1,1,punta arenas,-53.1500,-70.9167,11.06,87,75,1.54,CL,1708924608
2,2,uturoa,-16.7333,-151.4333,28.77,78,46,5.19,PF,1708924608
3,3,lib,8.3122,167.3766,27.69,81,22,6.67,MH,1708924609
4,4,daru,-9.0763,143.2092,30.16,74,57,1.49,PG,1708924609


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.7
)

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 15) &
                                (city_data_df["Humidity"] > 60) & (city_data_df["Humidity"] < 75) &
                                (city_data_df["Cloudiness"] <= 60) & (city_data_df["Wind Speed"] < 2)]

# Drop any rows with null values
ideal_cities = ideal_conditions.dropna(how='any')

# Display sample data
ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
328,328,hadibu,12.6500,54.0333,24.67,70,53,1.33,YE,1708924767
331,331,mukalla,14.5425,49.1242,24.41,71,1,1.17,YE,1708924497
479,479,van buren,35.4368,-94.3483,15.18,70,0,0.00,US,1708924872
557,557,la'ie,21.6477,-157.9253,21.98,71,24,1.79,US,1708924321


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
328,hadibu,YE,12.6500,54.0333,70,
331,mukalla,YE,14.5425,49.1242,71,
479,van buren,US,35.4368,-94.3483,70,
557,la'ie,US,21.6477,-157.9253,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hadibu - nearest hotel: No hotel found
mukalla - nearest hotel: فندق الجابري
van buren - nearest hotel: Holiday Inn Express Hotel & Suites Van Buren-Ft Smith Area
la'ie - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
328,hadibu,YE,12.6500,54.0333,70,No hotel found
331,mukalla,YE,14.5425,49.1242,71,فندق الجابري
479,van buren,US,35.4368,-94.3483,70,Holiday Inn Express Hotel & Suites Van Buren-F...
557,la'ie,US,21.6477,-157.9253,71,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.7,
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)